# Dataset del Titanic

El dataset del titanic es uno de los datasets más usados para aprender ML. Está publicado en Kaggle como una competencia para principantes: https://www.kaggle.com/c/titanic

En nuestro caso, lo usaremos para validar lo que hemos ido aprendiendo sobre aprendizaje supervisado y para ilustrar cómo trabajar con variables categóricas.

In [1]:
import pandas as pd
import numpy as np

from scipy.interpolate import spline
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
df = pd.DataFrame.from_csv("./train.csv")

C:\Users\a558859\AppData\Local\Continuum\anaconda3\envs\datahack\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [3]:
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Trabajando con datos categóricos.

A menudo, al entrenar modelos de ML, nos encontraremos con datos categóricos. Algunas familias de modelos, como los árboles de decisión, pueden trabajar directamente con estas categorías. Otros, como SVM, regresión lineal, entre otros, necesitan que hagamos algunas transformaciones. Afortunadamente, SKLearn nos permite hacer estas transformaciones fácilmente con LabelEncoder, LabelBinarizer y MultiLabelBinarizer.

## LabelEncoder

In [4]:
df["Pclass"].describe()

count    891.000000
mean       2.308642
std        0.836071
min        1.000000
25%        2.000000
50%        3.000000
75%        3.000000
max        3.000000
Name: Pclass, dtype: float64

In [5]:

le = LabelEncoder()

le.fit_transform(df["Pclass"])[0:10]

array([2, 0, 2, 0, 2, 2, 0, 2, 2, 1], dtype=int64)

In [6]:
df["Pclass"].unique()

array([3, 1, 2], dtype=int64)

In [7]:
df["Pclass"].describe()

count    891.000000
mean       2.308642
std        0.836071
min        1.000000
25%        2.000000
50%        3.000000
75%        3.000000
max        3.000000
Name: Pclass, dtype: float64

## Label Encoder

In [6]:

lb = LabelBinarizer()

lb.fit_transform(df["Sex"])[0:5]

array([[1],
       [0],
       [0],
       [0],
       [1]])

## MultiLabelBinarizer

In [9]:

ml = MultiLabelBinarizer()

ml.fit_transform([[sex] for sex in df["Sex"]])[0:10]

array([[0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0]])

In [10]:
set(df["Sex"])

{'female', 'male'}

# Ejercicio

Usando train.csv, entrena un modelo de ML para predecir si una persona sobrevivio el desastre del Titanic.

Pasos propuestos:

1. Haz una partición train/validation sobre el fichero train.csv, para quedarte con 70% de datos de entrenamiento y 20% de datos de validación.
2. Entrena un modelo sencillo usando únicamente clase. ¿Qué resultados obtienes en el set de validación?
3. Ahora entrena un modelo que busque predicir la supervivencia basándose en la clase y el género. ¿Qué resultados obtienes? ¿Son mejores que el modelo anterior?
4. Intenta generar características más interesantes que puedan ser útiles para explicar si una persona sobrevivio. Prueba modelos con esas características sobre el set de validación.

Pista: La edad puede ser particularmente útil. Recuerda: mujeres y niños primero...

5. Cuando hayas probado varios candidatos, prueba tu modelo sobre los datos de test.csv. ¿Cómo se comparan los resultados con el set de validación?

## 1.- Divido el conjunto df en 70% para train y 20% para test

Nos sobraría un 10%, es la intención usarlo como cross validation?

In [7]:
df_X = df.loc[:, df.columns != 'Survived']
df_Y = df.loc[:, df.columns == 'Survived']

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df_X, df_Y, train_size=0.7,test_size = 0.2, random_state=0)

In [11]:
X_train_1 = X_train.loc[:, ['Pclass']]
X_train_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 623 entries, 141 to 775
Data columns (total 1 columns):
Pclass    623 non-null int64
dtypes: int64(1)
memory usage: 9.7 KB


In [12]:
Y_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 623 entries, 141 to 775
Data columns (total 1 columns):
Survived    623 non-null int64
dtypes: int64(1)
memory usage: 9.7 KB


## 2.- Primer caso regresión usand PClass

### Regresion logarítmica

In [13]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression().fit(X_train_1, Y_train)

print("intercept b: {}".format(logreg.intercept_))
print("Log coeff: {}".format(logreg.coef_))

intercept b: [1.20588279]
Log coeff: [[-0.76916783]]


C:\Users\a558859\AppData\Local\Continuum\anaconda3\envs\datahack\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Regresion lineal

In [14]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression().fit(X_train_1, Y_train)

print("intercept b: {}".format(linreg.intercept_))
print("Linear coeff: {}".format(linreg.coef_))

intercept b: [0.80302762]
Linear coeff: [[-0.18622836]]


### Score de ambas regresiones

In [16]:
logreg.score(X_train_1, Y_train)

0.6773675762439807

In [15]:
linreg.score(X_train_1, Y_train)

0.10398325998740354

### Plot ambas regresiones

In [42]:
import matplotlib.pyplot as plt

In [46]:

# To plot the fitted logistic function as a smooth line
x_smooth = np.linspace(X_train.min(),X_train.max(),300)
y_log = 1
#/ (1+np.exp(-1*(X_train * logreg.coef_ + logreg.intercept_)))
y_log_smooth = spline(X_train[:,0], y_log, x_smooth)

# Linear regression fit
y_lin = X_C1 * linreg.coef_ + linreg.intercept_
y_lin_smooth = spline(X_train[:,0], y_lin, x_smooth)

plt.figure()
plt.title('Classification - one dimension - two groups')
plt.scatter(X_train, Y_train, c=Y_train, marker= 'o', s=50, cmap=cmap_bold)
plt.plot(x_smooth,y_log_smooth,color='blue')
plt.plot(x_smooth,y_lin_smooth,color="green")
plt.show()

TypeError: unhashable type: 'slice'

## 3.- Segundo caso regresión usando PClass y Sexo

In [17]:
X_train_2 = X_train.loc[:, ['Pclass','Sex']]
X_train_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 623 entries, 141 to 775
Data columns (total 2 columns):
Pclass    623 non-null int64
Sex       623 non-null object
dtypes: int64(1), object(1)
memory usage: 14.6+ KB


Codificamos en numérico la columna Sex

In [18]:
le = LabelEncoder()
X_train_2.loc[:,'Sex'] = le.fit_transform(X_train_2.loc[:,'Sex'])

In [19]:
X_train_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 623 entries, 141 to 775
Data columns (total 2 columns):
Pclass    623 non-null int64
Sex       623 non-null int64
dtypes: int64(2)
memory usage: 14.6 KB


### Creación y entrenamiento de los modelos

In [21]:
logreg2 = LogisticRegression().fit(X_train_2, Y_train)

print("intercept b: {}".format(logreg2.intercept_))
print("Log coeff: {}".format(logreg2.coef_))

intercept b: [2.58820294]
Log coeff: [[-0.75212555 -2.32713802]]


In [25]:
linreg2 = LinearRegression().fit(X_train_2, Y_train)

print("intercept b: {}".format(linreg2.intercept_))
print("Linear coeff: {}".format(linreg2.coef_))

intercept b: [1.04435367]
Linear coeff: [[-0.14584313 -0.50665818]]


### Score de ambas regresiones

In [23]:
logreg2.score(X_train_2, Y_train)

0.7849117174959872

In [26]:
linreg2.score(X_train_2, Y_train)

0.34553347565006687

## 4.- Buscar nuevas características

In [27]:
X_train["Age"].describe()

count    501.000000
mean      29.828004
std       14.728470
min        0.750000
25%       21.000000
50%       29.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [35]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 623 entries, 141 to 775
Data columns (total 10 columns):
Pclass      623 non-null int64
Name        623 non-null object
Sex         623 non-null object
Age         501 non-null float64
SibSp       623 non-null int64
Parch       623 non-null int64
Ticket      623 non-null object
Fare        623 non-null float64
Cabin       143 non-null object
Embarked    621 non-null object
dtypes: float64(2), int64(3), object(5)
memory usage: 53.5+ KB


Existen NA en la variable AGE, rellenémoslo con la media

In [43]:
X_train.loc[:,"Age"] = X_train.loc[:,"Age"].fillna(value=X_train.loc[:,"Age"].mean())

In [57]:
X_train.loc[:,"AgeRange"] = pd.cut(X_train.loc[:,"Age"],bins=[-np.inf,16,60,np.inf],labels=["Child", "Adult", "Elder"])

In [58]:
X_train.loc[:,'AgeRange'] = le.fit_transform(X_train.loc[:,'AgeRange'])

In [59]:
X_train_3 = X_train.loc[:, ['Pclass','Sex','AgeRange']]
X_train_3.loc[:,'Sex'] = le.fit_transform(X_train_3.loc[:,'Sex'])
X_train_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 623 entries, 141 to 775
Data columns (total 3 columns):
Pclass      623 non-null int64
Sex         623 non-null int64
AgeRange    623 non-null int64
dtypes: int64(3)
memory usage: 19.5 KB


### Creación y entrenamiento de los modelos

In [60]:
logreg3 = LogisticRegression().fit(X_train_3, Y_train)

print("intercept b: {}".format(logreg3.intercept_))
print("Log coeff: {}".format(logreg3.coef_))

intercept b: [2.54591334]
Log coeff: [[-0.74987474 -2.32983453  0.20718524]]


In [62]:
linreg3 = LinearRegression().fit(X_train_3, Y_train)

print("intercept b: {}".format(linreg3.intercept_))
print("Linear coeff: {}".format(linreg3.coef_))

intercept b: [1.03699032]
Linear coeff: [[-0.14516062 -0.50641527  0.03271383]]


### Score de ambas regresiones

In [63]:
linreg3.score(X_train_3, Y_train)

0.3464495365158531

In [61]:
logreg3.score(X_train_3, Y_train)

0.7849117174959872